In [1]:
import json

In [2]:
with open("./NanoAODv15.json") as f:
    data = json.load(f)

In [3]:
evt_content = data["Events Content"]

In [4]:
branches = evt_content.keys()

In [5]:
branches

dict_keys(['BeamSpot', 'CaloMET', 'CorrT1METJet', 'DST', 'Dataset', 'DeepMETResolutionTune', 'DeepMETResponseTune', 'Electron', 'FatJet', 'FatJetPFCand', 'FiducialMET', 'Flag', 'FsrPhoton', 'GenDressedLepton', 'GenIsolatedPhoton', 'GenJet', 'GenJetAK8', 'GenMET', 'GenPart', 'GenProton', 'GenVisTau', 'GenVtx', 'Generator', 'HLT', 'HLTriggerFinalPath', 'HLTriggerFirstPath', 'HTXS', 'IsoTrack', 'Jet', 'L1', 'L1Reco', 'L1simulation', 'LHE', 'LHEPart', 'LHEPdfWeight', 'LHEReweightingWeight', 'LHEScaleWeight', 'LHEWeight', 'LowPtElectron', 'MC', 'Muon', 'OtherPV', 'PFCand', 'PFMET', 'PSWeight', 'PV', 'PVBS', 'Photon', 'Pileup', 'PuppiMET', 'RawPFMET', 'RawPuppiMET', 'Rho', 'SV', 'SoftActivityJet', 'SoftActivityJetHT', 'SoftActivityJetHT10', 'SoftActivityJetHT2', 'SoftActivityJetHT5', 'SoftActivityJetNjets10', 'SoftActivityJetNjets2', 'SoftActivityJetNjets5', 'SubGenJetAK8', 'SubJet', 'Tau', 'TauProd', 'TauSpinner', 'TrackGenJetAK4', 'TrigObj', 'TrkMET', 'boostedTau', 'bunchCrossing', 'event'

In [6]:
used_branch =['Electron',
            'FatJet', 'FatJetPFCand','Flag', 'FsrPhoton', 'GenDressedLepton', 'GenIsolatedPhoton', 'GenJet', 'GenJetAK8', 'GenMET', 'GenPart',
            'GenVisTau', 'GenVtx', 'Generator', 
            'IsoTrack', 'Jet', 
              'LHE', 'LHEPart', 'LHEPdfWeight', 'LHEReweightingWeight', 'LHEScaleWeight', 'LHEWeight', 'LowPtElectron', 
                'Muon', 'OtherPV', 'PFCand', 'PFMET', 'PSWeight', 'PV', 'Photon', 'Pileup', 'PuppiMET',
                  'Rho', 'SV', 
                  'SubGenJetAK8', 'SubJet', 'Tau', 'TauProd', 'TauSpinner', 'TrackGenJetAK4', 'TrigObj', 'TrkMET', 'boostedTau', 'bunchCrossing', 'event', 'genTtbarId', 'genWeight', 'luminosityBlock', 'orbitNumber', 'run']

In [7]:
def is_array(branch, evt_content):
    leaf_list = evt_content
    if any(leaf.startswith(f"n{branch}") for leaf in leaf_list[branch]):
        return True
    else:
        return False

def process_branch(branch, evt_content):
    total_string = f"// {branch}\n"
    
    for leaf in evt_content[branch]:

        if leaf == f"n{branch}" and is_array(branch, evt_content):
            total_string = total_string + f"std::unique_ptr<TTreeReaderValue<{evt_content[branch][leaf]['type']}>> {leaf}_Ptr;"
            total_string = total_string + "\n"
            total_string = total_string + f"{evt_content[branch][leaf]['type']} &{leaf};"
        elif is_array(branch, evt_content):
            total_string = total_string + f"std::unique_ptr<TTreeReaderArray<{evt_content[branch][leaf]['type']}>> {leaf}_Ptr;"
            total_string = total_string + "\n"
            total_string = total_string + f"TTreeReaderArray<{evt_content[branch][leaf]['type']}> &{leaf};"
        else:
            total_string = total_string + f"std::unique_ptr<TTreeReaderValue<{evt_content[branch][leaf]['type']}>> {leaf}_Ptr;"
            total_string = total_string + "\n"
            total_string = total_string + f"{evt_content[branch][leaf]['type']} {leaf};"
        total_string = total_string + "\n"
    return total_string

In [8]:
def process_branch(branch, evt_content):
    total_string = f"// {branch}\n"
    for leaf in evt_content[branch]:
        if leaf == f"n{branch}" and is_array(branch, evt_content):
            total_string = total_string + f"TTreeReaderValueWrapper<{evt_content[branch][leaf]['type']}> {leaf};"
        elif is_array(branch, evt_content):
            total_string = total_string + f"TTreeReaderArrayWrapper<{evt_content[branch][leaf]['type']}> {leaf};"
        else:
            total_string = total_string + f"TTreeReaderValueWrapper<{evt_content[branch][leaf]['type']}> {leaf};"

        total_string = total_string + "\n"
    return total_string

In [9]:
import os
os.remove("header.h")
for branch in branches:
    #clear the file
    if not branch in used_branch:
        continue
    
    with open(f"header.h", "a") as f:
        f.write(process_branch(branch, evt_content))
        f.write("\n")
    

In [37]:
def process_branch_src(branch, evt_content):
    total_string = f"// {branch}\n"
    for leaf in evt_content[branch]:
        if leaf == f"n{branch}" and is_array(branch, evt_content):
            total_string = total_string + f"{leaf}_Ptr = std::make_unique<TTreeReaderValue<{evt_content[branch][leaf]['type']}>>(*fReader, \"{leaf}\");"
            total_string = total_string + "\n"
            total_string = total_string + f"{leaf} = **{leaf}_Ptr;"
        elif is_array(branch, evt_content):
            total_string = total_string + f"{leaf}_Ptr = std::make_unique<TTreeReaderArray<{evt_content[branch][leaf]['type']}>>(*fReader, \"{leaf}\");"
            total_string = total_string + "\n"
            total_string = total_string + f"{leaf} = *{leaf}_Ptr;"
        else:
            total_string = total_string + f"{leaf}_Ptr = std::make_unique<TTreeReaderValue<{evt_content[branch][leaf]['type']}>>(*fReader, \"{leaf}\");"
            total_string = total_string + "\n"
            total_string = total_string + f"{leaf} = **{leaf}_Ptr;"

        total_string = total_string + "\n"
    return total_string

In [ ]:
def process_branch_src(branch, evt_content):
    total_string = f"// {branch}\n"
    for leaf in evt_content[branch]:
        if leaf == f"n{branch}" and is_array(branch, evt_content):
            total_string = total_string + f"{leaf}.init(fReader, \"{leaf}\");"
            total_string = total_string + "\n"
            total_string = total_string + f"{leaf} = **{leaf}_Ptr;"
        elif is_array(branch, evt_content):
            total_string = total_string + f"{leaf} = new TTreeReaderArray<{evt_content[branch][leaf]['type']}>(*fReader, \"{leaf}\");"
        else:
            total_string = total_string + f"{leaf}_Ptr = new TTreeReaderValue<{evt_content[branch][leaf]['type']}>(*fReader, \"{leaf}\");"
            total_string = total_string + "\n"
            total_string = total_string + f"{leaf} = **{leaf}_Ptr;"
        total_string = total_string + "\n"
    return total_string

In [38]:
import os
if os.path.exists("src.cc"):
    os.remove("src.cc")
for branch in branches:
    if not branch in used_branch:
        continue
    #clear the file
    with open(f"src.cc", "a") as f:
        f.write(process_branch_src(branch, evt_content))
        f.write("\n")


In [20]:
def process_branch_src(branch, evt_content):
    total_string = f"// {branch}\n"
    for leaf in evt_content[branch]:
        if leaf == f"n{branch}" and is_array(branch, evt_content):
            total_string = total_string + f"delete {leaf}_Ptr;"
        elif is_array(branch, evt_content):
            total_string = total_string + f"delete {leaf};"
        else:
            total_string = total_string + f"delete {leaf}_Ptr;"
        total_string = total_string + "\n"
    return total_string

In [21]:
import os
if os.path.exists("src_del.cc"):
    os.remove("src_del.cc")
for branch in branches:
    if not branch in used_branch:
        continue
    #clear the file
    with open(f"src_del.cc", "a") as f:
        f.write(process_branch_src(branch, evt_content))
        f.write("\n")
